In [8]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
import torch

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


All the experiment return a pandas dataframe. 

    - mem: stand for the factual recall mechanism
    - cp: stand for the counterfactual mechanism   

## Load data and model

In [10]:
from dataset import BaseDataset
from model import ModelFactory
model = ModelFactory.create("gpt2", device="mps")
dataset = BaseDataset(
    path= "../data/full_data_sampled_gpt2_with_subjects.json",
    model=model,
    experiment="copyVSfact",
    premise="Redefine",
    start=0,
    end=15
)

Loaded pretrained model gpt2 into HookedTransformer


Tokenizing and computing lengths: 100%|██████████| 15/15 [00:00<00:00, 307.40it/s]


## Logit Lens
Compute the logit lens in the residual stream

In [4]:
from experiment import LogitLens
logit_lens = LogitLens(
    dataset=dataset,
    model=model,
    batch_size=40,
    experiment="copyVSfact",
)

In [5]:
logit_lens.run(
    component="resid_post", # "resid_pre" (pre attention) or "resid_mid" (post attention and pre mlp) or "resid_post" (post mlp)
    normalize_logit="none"
)

Logit lens:: 100%|██████████| 5/5 [00:16<00:00,  3.39s/it]


,component,layer,position,mem,cp,diff,mem_perc,cp_perc,diff_perc,mem_std,cp_std,mem_idx,cp_idx
0,resid_post,0,0,-0.221067,-0.416839,0.195772,-120.440071,-164.294479,-54.809231,0.552207,0.385051,None,None
1,resid_post,0,1,4.174799,1.038706,3.136094,286.005371,60.213051,623.917847,1.648587,1.051461,None,None
2,resid_post,0,2,0.665571,0.150635,0.514935,-38.460789,-76.765541,18.864695,0.737493,0.647751,None,None
3,resid_post,0,3,0.554917,0.128475,0.426441,-48.691956,-80.183578,-1.562744,0.851559,0.625169,None,None
4,resid_post,0,4,0.209353,0.024318,0.185036,-80.643013,-96.249153,-57.287453,0.262376,0.332149,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,resid_post,11,9,8.206142,5.814023,2.392119,19.579527,9.388631,54.581394,1.804289,1.956465,None,None
164,resid_post,11,10,3.649913,1.921111,1.728802,-46.813637,-63.855030,11.717087,3.328378,2.535936,None,None
165,resid_post,11,11,7.962111,5.874662,2.087448,16.023512,10.529538,34.893211,1.500016,1.428558,None,None
166,resid_post,11,12,9.162442,7.422668,1.739773,33.514683,39.654682,12.426073,2.791573,2.762402,None,None


## Logit Attribution

Compute the logit lens for the output of attention blocks, MLP blocks and attention heads

In [ ]:
from experiment import LogitAttribution
logit_attribution = LogitAttribution(
    dataset=dataset,
    model=model,
    batch_size=40,
    experiment="copyVSfact",
)

In [ ]:
logit_attribution.run()

## Extract Pattern heads

In [ ]:
from experiment import HeadPattern

head_pattern = HeadPattern(
    dataset=dataset,
    model=model,
    batch_size=40,
    experiment="copyVSfact",
)

In [ ]:
head_pattern.run()